# Image ==> Matrix
Given Image Address in Share Repository, Generate Geometry Output as Matrices. \
Split into binary matrix and RI matrix, processes into RI matrix only for cell geometry.

In [1]:
# Import libraries:
import numpy as np
from PIL import Image 
import tiffile as tf
import scipy as sp
import os

## Set-up I/O Path Allocations 
Declare input data path and access tiff or tif. \
The path variable can be changed to output different datas' geometry arrays.

In [2]:
# Path Variables:
repo_path = "/Users/caozehuang/Documents/GitHub/Photon-Transport-Simulation/"     # Replace with repository path
export_address = "data/extracted_ri/"
access_address = "data/processed/"         

# Helpers:
def output_address(subfolder="data/", repository=repo_path):
    out_path = f"{repository}{subfolder}"
    return out_path

def modified_output_address(file_name):
    name_part = file_name.split(".")[0]
    new_name = f"{name_part}_extracted.mat"
    return new_name

In [8]:
# Accessing the file:
file_name = "Test.tif"     # Replace with actual file name
access_path = f"{repo_path}{access_address}{file_name}"
print("Accessing: " + access_path)

Accessing: /Users/caozehuang/Documents/GitHub/Photon-Transport-Simulation/data/processed/Test.tif


Here, we load and generate Numpy arrays to store our pixel matrices.
1. Raw Matrix(from fiji)
2. RI Matrix(normalized to standard notation)

In [9]:
# Load raw file:
if(file_name.split(".")[-1] == "tiff" or file_name.split(".")[-1] == "tif"):
    raw = tf.imread(access_path)
    print(f"TIFF loaded successfully!")
elif(file_name.split(".")[-1] == "jpeg" or file_name.split(".")[-1] == "jpg" or file_name.split(".")[-1] == "png"):
    raw = Image.open(access_path)
    print(f"Image loaded successfully!")
else:
    raise ValueError("Unsupported file format. Please use TIFF, JPEG, or PNG.")

TIFF loaded successfully!


In [10]:
# Convert to numpy array (already a numpy matrix)
try:
    raw_mat = np.array(raw)
except Exception as e:
    print(f"Error converting image to numpy array: {e}")
    raise

# Print raw file information:
print("Raw file information:")
print(f"Shape: {raw_mat.shape}")
print(f"Data type: {raw_mat.dtype}")
print("\n")

# Fix RI values of raw Fiji Output to be in the range of 1.0 to 2.0:
ri_mat = raw_mat / 10000
print("Refractive Index (RI) matrix information:")
print(f"Shape: {ri_mat.shape}")
print(f"Data type: {ri_mat.dtype}")
print(f"Min value: {ri_mat.min()}, Max value: {ri_mat.max()}")
print(ri_mat)  # Print the RI matrix to verify its contents

Raw file information:
Shape: (212, 376)
Data type: uint16


Refractive Index (RI) matrix information:
Shape: (212, 376)
Data type: float64
Min value: 1.3244, Max value: 1.4041
[[1.3325 1.3325 1.3324 ... 1.3326 1.3326 1.3326]
 [1.3323 1.332  1.332  ... 1.3325 1.3326 1.3325]
 [1.3331 1.3329 1.3324 ... 1.3324 1.3325 1.3328]
 ...
 [1.3331 1.3333 1.3331 ... 1.3324 1.3325 1.3328]
 [1.3342 1.3339 1.333  ... 1.3322 1.3325 1.3334]
 [1.3336 1.3335 1.3328 ... 1.3325 1.3326 1.3331]]


# Extraction and Processing
1. Move to process the RI tiff file, return binary mask of the RI matrix. 
2. Overlay binary mask onto the RI matrix to retrieve only the RI of the cell body(assume it has been properly filtered). 
3. Return the matrix that only contain the cell body's RI


In [17]:
# Apply RI Threshold to mask the image:
ri_threshold = 1.34     # Can be changed depending on cell RI range and desired masking

# Create binary matrix based on RI threshold
binary_mat = np.where(ri_mat >= ri_threshold, 1, 0)
print(f"Binary matrix created with threshold {ri_threshold}.\n")
print(f"Shape: {binary_mat.shape}\n")
print(f"Unique values in binary matrix: {np.unique(binary_mat)}\n")

Binary matrix created with threshold 1.34.

Shape: (212, 376)

Unique values in binary matrix: [0 1]



In [19]:
# Makes Matrices are dimensionally compatible
if ri_mat.shape != binary_mat.shape:
    raise ValueError("RI matrix and binary matrix have different shapes. Please check the input data.")

# Apply binary mask to RI matrix
masked_ri_mat = binary_mat * ri_mat

# Print masked RI matrix information:
print("Masked RI matrix created by overlaying binary mask onto RI matrix.\n")
print(f"Shape: {masked_ri_mat.shape}\n")
print(f"Data type: {masked_ri_mat.dtype}\n")
print(f"Max value: {masked_ri_mat.max()}\n")
print(masked_ri_mat) # Print the masked RI matrix to verify its contents

Masked RI matrix created by overlaying binary mask onto RI matrix.

Shape: (212, 376)

Data type: float64

Max value: 1.4041

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Saving

In [20]:
# Save the extracted geometry as a .mat file:
# Export address and name:
output_path = output_address(export_address, repo_path)
output_name = modified_output_address(file_name=file_name)

# Save
try:
    sp.io.savemat(f"{output_path}{output_name}", {"masked_ri": masked_ri_mat})
    print(f"Extracted RI matrix saved successfully at {output_path}{output_name}!")
except Exception as e:
    print(f"Error saving extracted RI matrix: {e}")
    raise


Extracted RI matrix saved successfully at /Users/caozehuang/Documents/GitHub/Photon-Transport-Simulation/data/extracted_ri/Test_extracted.mat!
